<a href="https://colab.research.google.com/github/skj092/Computer_Vision_Lab/blob/main/experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

# moving kaggle.json to /root/.kaggle
mv kaggle.json /root/.kaggle 

# downloading the dataset
kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

# unzipping the dataset
unzip /content/dogs-vs-cats-redux-kernels-edition.zip 

# unzipping train and test 
unzip train.zip 

unzip test.zip 

In [2]:
# importing the required libraries
import os 
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, Subset 
from glob import glob
from PIL import Image
from torchvision import transforms


# creating the dataset class
class DogsVsCatsDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        label = 1 if 'dog' in image_path else 0
        return image, label

# transform 
tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# defining the training function 
from tqdm import tqdm
import torch



def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    train_loss, val_loss, train_acc, val_acc = [], [], [], []
    for epoch in range(num_epochs):
        model.train()
        for xb, yb in tqdm(train_loader):
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            yb_ = model(xb)
            loss = criterion(yb_, yb)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            train_acc.append((yb_.argmax(1) == yb).float().mean())
        model.eval()
        with torch.no_grad():
            for xb, yb in tqdm(val_loader):
                xb, yb = xb.to(device), yb.to(device)
                yb_ = model(xb)
                loss = criterion(yb_, yb)
                val_loss.append(loss.item())
                val_acc.append((yb_.argmax(1) == yb).float().mean())
        print(f'Epoch: {epoch+1}, Train Loss: {torch.tensor(train_loss).mean():.4f}, Train Accuracy: {torch.tensor(train_acc).mean():.4f}, Val Loss: {torch.tensor(val_loss).mean():.4f}, Val Accuracy: {torch.tensor(val_acc).mean():.4f}')
        


In [6]:
import os
import torch.nn as nn 
import torch, torchvision 
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from glob import glob
from pathlib import Path


# defining the hyperparameters
batch_size = 32
num_epochs = 10
learning_rate = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# path to the dataset
path = Path('train')

# getting the list of images
images = glob(os.path.join(path, '*.jpg'))

# splitting the dataset into train and validation
train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

# creating the dataset objects
train_dataset = DogsVsCatsDataset(train_images, transform=tfms)
val_dataset = DogsVsCatsDataset(val_images, transform=tfms)

# creating subset of the dataset with 50% of the data
train_subset = Subset(train_dataset, indices=range(0, len(train_dataset), 2))
valid_subset = Subset(val_dataset, indices=range(0, len(val_dataset), 2))

# creating the dataloaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(valid_subset, batch_size=32, shuffle=False)

# printing the length of the dataset
print('Length of the train dataset: ', len(train_dataset))
print('Length of the validation dataset: ', len(val_dataset))
print('Length of the train subset: ', len(train_subset))
print('Length of the validation subset: ', len(valid_subset))

# model
model = Model()
model.to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Length of the train dataset:  20000
Length of the validation dataset:  5000
Length of the train subset:  10000
Length of the validation subset:  2500


In [14]:
# pretrained alexnet model
from torchvision import models 

# pretrained alexnet model with freezing the weights 
class AlexNetPretrained(nn.Module):
    def __init__(self):
        super(AlexNetPretrained, self).__init__()
        self.model = models.alexnet(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.classifier[6] = nn.Linear(4096, 2)

    def forward(self, x):
        x = self.model(x)
        return x

model = AlexNetPretrained()
model.to(device)

AlexNetPretrained(
  (model): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)

In [15]:
# training the model
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)

100%|██████████| 79/79 [00:12<00:00,  6.32it/s]


Epoch: 1, Train Loss: 0.1667, Train Accuracy: 0.9363, Val Loss: 0.1255, Val Accuracy: 0.9513


100%|██████████| 79/79 [00:12<00:00,  6.42it/s]


Epoch: 2, Train Loss: 0.1514, Train Accuracy: 0.9433, Val Loss: 0.1369, Val Accuracy: 0.9515


100%|██████████| 79/79 [00:12<00:00,  6.48it/s]


...

Epoch: 8, Train Loss: 0.1207, Train Accuracy: 0.9574, Val Loss: 0.1622, Val Accuracy: 0.9514


100%|██████████| 79/79 [00:12<00:00,  6.46it/s]


Epoch: 9, Train Loss: 0.1187, Train Accuracy: 0.9584, Val Loss: 0.1631, Val Accuracy: 0.9516


100%|██████████| 79/79 [00:12<00:00,  6.39it/s]


Epoch: 10, Train Loss: 0.1175, Train Accuracy: 0.9592, Val Loss: 0.1646, Val Accuracy: 0.9519
